# Bradley-Terry's Model

前回の勉強会で、$\chi^2$(カイ二乗)検定を学びました。

あるモデルから計算される理論値と実測値の差を$\chi^2$分布に帰着させることで、モデルに適合してるか否かを検定できます。
その一番使われる例として分割表 (contingency table) について学びました。(分割法の検定は役に立つのでよく復習しておいてください)

今回は分割表以外での$\chi^2$検定の話として、Bradley-Terryモデルの話をします。

Bradley-Terry モデルは、比較結果を予測するためのモデルです。比較の具体例として、今回の演習課題としては野球の勝敗を例にとります。

2014年度セ・リーグ対戦結果(http://bis.npb.or.jp/2014/stats/std_c.html を参照しました)は下記のようになります。
数値は勝った試合数(引き分けは0.5勝扱い)です。


|  　  | 対読 | 対阪 | 対広 | 対中 | 対De | 対ヤ | 総勝数 |
|:---:|:---:|:----:|:---:|:---:|:---:|:----:|:----:|
| 読売 | *   | 13   | 13.5 | 16 | 11  | 13   | 82.5 |
| 阪神 | 11  | *    | 14   | 12.5 | 16 | 13 | 75.5 |
| 広島 | 10.5 | 10  | *   | 14 | 15.5 | 16 | 75 | 
| 中日 | 8 | 11.5 | 10 | * | 14.5 | 11.5 | 69 |
| DeNA | 13 | 8 | 8.5 | 9.5 | * | 16 | 68 |
| ヤクルト | 11 | 11 | 8 | 12.5 | 8 | * | 61.5 |

Bradley-Terry モデルではチーム$i$がチーム$j$に勝つ確率$p_{ij} = p(i>j)$が、
$$p_{ij} = \frac{\theta_i}{\theta_i + \theta_j}$$
で表現できる、というモデルです。$\theta_i$はチーム$i$の「強さ」を表現する一次元的な数値であると考えることが出来ます。

もう少しモデルの数値的な意味を考えてみましょう。$p_{ij}$ のオッズ比の対数を考えます。(確率 $p$ に対して $\frac{p}{1-p}$ を $p$ のオッズ比といいます。ロジスチック回帰分析でよく出てくる用語です)

この量が、チーム$i$に依存する量$\beta_i$ とチーム$j$に依存する量 $\beta_j$ の差で書けると仮定すると、
$$\log \frac{p_{ij}}{1-p_{ij}} = \log \frac{p_{ij}}{p_{ji}} = \beta_i - \beta_j$$
ここで、$\exp \beta_i = \theta_i$ と置くと上の式が出ます。

$\theta_i$ をどのように計算するかですが、(導出の都合上、簡単のために引き分けがないと仮定すると)チーム$i$がチーム$j$に勝った勝ち数$w_{ij}$の分布は、
$$P(w_{ij}) = \binom{w_{ij}+w_{ji}}{w_{ij}}p_{ij}^{w_{ij}}p_{ji}^{w_{ji}}$$
これを全組み合わせについて積を考えると尤度関数
$$\mathcal{L}(\mathbf{p}) = \prod_{1 \leq i < j \leq n} \binom{r_{ij}}{w_{ij}}p_{ij}^{w_{ij}}p_{ji}^{w_{ji}}, \;\; r_{ij} = w_{ij}+w_{ji}$$
が得られ、これを解くと（導出は、例えば「自然科学の統計学 p.168」（東大出版会）参照）、最尤解$\hat{\theta_i}$は
$$\hat{\theta_i}\sum_{j \neq i} \frac{r_{ij}}{\hat{\theta_i}+\hat{\theta_j}} = \sum_{j \neq i}w_{ij}$$
で得られます。ただし、$\theta_i$はお互いの比にしか意味がないため、不定性を消すために
$$\sum_i \hat{\theta_i} = 1$$
と定めることにします。

この方程式は下記のような反復法で解くことが出来ます。

1. 初期値を $\theta_i^0 = \frac{1}{6}, \; (i=1, \cdots, 6)$ とする。
2. 現在の近似値を元に最尤解を解く。
$$\tilde{\theta}_i^{t+1} = \frac{\sum_{j \neq i} w_{ij}}{\sum_{j \neq i}\frac{r_{ij}}{\theta_i^t + \theta_j^t}}$$
3. 得られた最尤解をスケーリングする。
$$\theta_i^{t+1} = \frac{\tilde{\theta}_i^{t+1}}{\sum_i \tilde{\theta}_i^{t+1}}$$

### 課題：

上に書かれた反復法を用いて、上記のリーグ戦のデータに対して $\theta_i \; (i=1,\cdots,6)$ を求めよ。

試合の総数 $r_{ij} = w_{ij} + w_{ji} = 24$ であり、$\sum_{j \neq i} w_{ij}$ はチーム$i$の総勝数であることを用いよ。

適当な閾値を用いて収束判定を行っても構わないし、適当にループを100回回す、といういい加減な方法でも構わない。(Excelのソルバーなどを用いても簡単に解ける)

解答は下記のような値になるはずである。

`[ 0.21098976  0.17900355  0.17692579  0.15381943  0.15026899  0.12899249]`

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
theta = np.ones(6)/6.0 # \theta 初期値
t = np.zeros(6) # ワークエリア \tilde{\theta}
r = 24.0 #この例ではたまたますべての同じというだけ #対戦回数は記録しておく必要がある。
ws = np.array([82.5, 75.5, 75.0, 69.0, 68.0, 61.5])  # 総勝数 \sum_{j \neq i} w_{ij}
for iloop in range(0, 100):
    for i in range(0,6):
        acc = 0 # 総和用のアキュムレータ
        for j in range(0,6):
            if (j == i):
                pass
            else:
                acc += r / (theta[i]+theta[j])
        t[i] = ws[i]/acc
    s = sum(t)
    for i in range(0, 6):
        theta[i] = t[i] / s
print(theta)

[0.21098976 0.17900355 0.17692579 0.15381943 0.15026899 0.12899249]


求めた $\theta_i$ の値を用いて、Bradley-Therry モデルに基づく各試合の結果 `calc_win` を計算すると下記のようになります。

In [3]:
# 先の問題をスキップした人は下記のコメントを外して解け
#theta = np.array([ 0.21098976, 0.17900355, 0.17692579, 0.15381943, 0.15026899, 0.12899249])

acc = []
for i in range(0, 5):
    t = theta[i+1:]
    acc.append(24*theta[i]/(theta[i]+t))
calc_win = np.array(acc)
calc_win

array([array([12.98420798, 13.05375423, 13.88055576, 14.01697344, 14.89417256]),
       array([12.07005076, 12.9080187 , 13.04720172, 13.94850806]),
       array([12.83833825, 12.977649  , 13.88023954]),
       array([12.1401082 , 13.05343255]), array([12.91426143])],
      dtype=object)

実際の試合の結果 `real_win` は下記の通りです。

In [4]:
real_win = np.array([np.array([13.0, 13.5, 16.0, 11.0, 13.0]),
                     np.array([14.0, 12.5, 16.0, 13.0]),
                     np.array([14.0, 15.5, 16.0]),
                     np.array([14.5, 11.5]),
                     np.array([16.0])])


### 課題

`calc_win` と `real_win` を用いて
$$\chi^2 = \sum_{1 \leq i < j \leq 6} \frac{(\text{実測値}-\text{理論値})^2}{\text{理論値}}$$
を計算しなさい。

この問題での$\chi^2$分布の自由度は10です。その理由を考察しなさい。

$\chi^2$検定を行い、Bradley-Terry モデルが適合しているかを検定しなさい。(`scipy.stats.chi2`の`chi2.isf(0.05, 10)`にて $\chi^2_{0.05}(10)$ の値を得られます。詳しくは前回を復習)

確率変数は $1+2+3+4+5=15$ 個。モデルの「自由な」パラメータ数は $6-1$ 個。よって自由度は$15-5=10$個。
計算した $\chi^2 = 4.58$ であり、$\chi^2_{0.05}(10) = 18.3$ であるから、有意水準 5% では棄却されない。
それなりの一致をしているといえる。

In [5]:
x2 = (calc_win - real_win)**2/calc_win
print(x2)
acc = 0.0
for i in range(0,5):
    acc += sum(x2[i])
print(acc)
from scipy.stats import chi2
print(chi2.isf(0.05, 10))

[array([1.92070075e-05, 1.52550204e-02, 3.23621329e-01, 6.49364769e-01,
       2.40892179e-01])
 array([0.30859059, 0.01289735, 0.66826726, 0.0644992 ])
 array([0.10511158, 0.49024709, 0.32372528])
 array([0.45873473, 0.18486729]) array([0.73730755])]
4.583400416157484
18.30703805327515


### 最後に

Bradley-Terryモデルはリーグ戦の結果を予測する以外にも例えば、一対比較法による官能検査（被験者にサンプルを同時に２つ与えて比較させる）などにも使用可能であり、もしBradle-Terryモデルが適合するならば、ユーザの好みの順序関係を何かのパラメータ一つで近似できるということになります。

なお、Bradle-Terryモデルを仮定するということは、
$$p_{ij} p_{jk} p_{ki} = \frac{\theta_i}{\theta_i + \theta_j} \frac{\theta_j}{\theta_j + \theta_k}
\frac{\theta_k}{\theta_k + \theta_i} = p_{ji} p_{ik} p_{kj}$$
となります。
仮に $i > j > k > i$ のような３すくみ関係が存在すると、$p_{ij} p_{jk} p_{ki} > p_{ji} p_{ik} p_{kj}$ となるから、Bradley-Terry モデルは、３すくみ関係が無い、と仮定しているともいえます。
